# Step 1, DataJoin: To clean and merge 311 service request data with the NYC Street Tree Census
** Note that throughout this and other jupyter notebooks, the following terminology is used:
- falls: refer to fallen branches (from the 311 service request dataset) since 2015
- service requests: refer to 311 service requests for damaged trees or overhanging branches (but NOT fallen branches) since 2015
- tree census: refers to the 2015 NYC Street Tree Census

Packages:

In [2]:
# Import packages
import pandas as pd
import numpy as np
import geopandas
import math

Functions

In [3]:
''' Function to match a 311 service request tree with a 2015 Street Tree Census tree_id
        input is the address, latitude, and longitude of the 311 service request and the Street Tree Census dataframe
        output is the tree_id '''

# the function
def assign_tree_id(address,lat,lon,df):
    # Find all street trees assigned to the same address as the 311 service request
    tempdf = df[df.address==address]
    # If multiple street trees are assigned to the address, find the one closest to the service request by latitude-longitude distance and output its tree_id
    if tempdf.count()['tree_id']>=2:
        dis=np.sqrt(np.square(tempdf.latitude-lat)+np.square(tempdf.longitude-lon))
        tree_index = np.argmin(np.array(dis))
        return tempdf.tree_id.iloc[tree_index]
    # If no street trees have the address output nan
    elif tempdf.count()['tree_id']==0:
        return np.nan
    # If only one street tree has the address, output that tree_id
    else:
        return tempdf.tree_id.to_numpy()[0]

### First, import and organize the 2015 NYC Street Tree Census data.
The tree census data is stored in a GIS shapefile, so it needs to be imported as a geopandas geodataframe and converted to a pandas dataframe.

Many of the columns can be dropped, like political boundaries (e.g. state assembly district), information about the data entry, and tree specifics (e.g. species latin name).

In [4]:
# Import tree census data as a geopandas geodataframe
tree_census_geodf = geopandas.read_file('Data/Trees/2015 Street Tree Census - Tree Data/geo_export_f9cac9c3-66c4-45b6-899b-823a270d8754.shp')

# Convert tree census data into a pandas dataframe, remove un-needed columns, and remove geometry columns
tree_census_df = pd.DataFrame(tree_census_geodf.drop(columns='geometry'), copy=True)
tree_census_df.drop(['boro_ct', 'borocode', 'boroname', 'brnch_ligh',
       'brnch_othe', 'brnch_shoe', 'cb_num', 'cncldist','nta',
       'nta_name','spc_latin', 'st_assem', 'st_senate', 'state','stump_diam','user_type', 'x_sp', 'y_sp', 'zip_city'],axis=1,inplace=True)

### Next, import 311 Tree fall service requests and assign each a street tree census tree_id
Several unneeded location columns can be dropped from the 311 requests.

Only falls after the tree census date (2015) will be used.  Only branch falls, not entire tree falls, will be used.  All requests for trees in parks (not street trees) will be removed.

In [12]:
# Import 311 tree fall service request data and drop unneeded columns
fall_df = pd.read_csv('Data/311/Tree_Fall_311.csv',quoting=3)
fall_df.drop([
       'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Status', 'Borough',
       'X Coordinate (State Plane)', 'Y Coordinate (State Plane)'],axis=1, inplace=True)

# Remove all falls before 2015 (before the street tree census)
fall_df['Created Date'] = pd.to_datetime(fall_df['Created Date'])
fall_df = fall_df.where(fall_df['Created Date'] > '2015-01-01')

# Remove all falls that occured in parks, and not in street trees
fall_df = fall_df.where(fall_df['Location Type'] != 'Park')

# Only keep branch falls (eliminate whole tree falls)
fall_df=fall_df[fall_df.Descriptor=='Branch or Limb Has Fallen Down']

Match each 311 branch fall service request with a street tree census tree_id using the assign_tree_id function defined above.  This matches trees based on address and lat-lon distance.

In [14]:
# In order to match service request tree falls to street tree census tree_ids, remove service requests without addresses or coordinates
fall_df = fall_df[fall_df['Incident Address'].isna()==False]
fall_df = fall_df[fall_df.Latitude.isna()==False]
fall_df = fall_df[fall_df.Longitude.isna()==False]

In [26]:
# Find closest tree from the street tree census and then assign its tree_index to the 311 data frame
fall_tree_ids_list = [assign_tree_id(address,lat,lon,tree_census_df.copy()) for address,lat,lon in zip(fall_df['Incident Address'],fall_df['Latitude'],fall_df['Longitude'])]
fall_df['tree_id']=fall_tree_ids_list

In [27]:
# save fallen tree ids so I don't need to run that function again
fall_df.to_csv('Data/Intermediate/tree_falls_with_id.csv', index=False)

In [28]:
# export id columns too, just in case
tid_fall_df = pd.DataFrame(fall_df, columns=['tree_id'])
tid_fall_df.to_csv('Data/Intermediate/fall_tree_id.csv', index=False)

### Next, import 311 tree damage and overhanging tree service requests and assign each a street tree census tree_id
Follow the same process as above with tree falls.  Again only use data from before 2015.  Again remove all requests for trees in parks (not street trees).

In [5]:
# Import and organize 311 tree damage and overhanging branch service requests and drop unneeded columns
service_request_df = pd.read_csv('Data/311/Tree_Service_Requests_311.csv',quoting=3)
service_request_df.drop([
       'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Status', 'Borough',
       'X Coordinate (State Plane)', 'Y Coordinate (State Plane)'],axis=1,inplace=True)

# Remove all service requests before 2015 (before the street tree census)
service_request_df['Created Date'] = pd.to_datetime(service_request_df['Created Date'])
service_request_df = service_request_df.where(service_request_df['Created Date'] > '2015-01-01')

# Remove all falls that occured in parks, and not in street trees
service_request_df = service_request_df.where(service_request_df['Location Type'] != 'Park')

C:\Users\brian\.conda\envs\insight\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Match each 311 branch fall service request with a street tree census tree_id using the assign_tree_id function defined above.  This matches trees based on address and lat-lon distance.

In [6]:
# In order to match service requests to street tree census tree_ids, remove service requests without addresses or coordinates
service_request_df = service_request_df[service_request_df['Incident Address'].isna()==False]
service_request_df = service_request_df[service_request_df.Latitude.isna()==False]
service_request_df = service_request_df[service_request_df.Longitude.isna()==False]

In [ ]:
# Find closest tree from the street tree census and then assign its tree_index to the 311 data frame
tree_ids_service_requests = [assign_tree_id(address,lat,lon,tree_census_df.copy()) for address,lat,lon in zip(service_request_df['Incident Address'],service_request_df.Latitude,service_request_df.Longitude)]
service_request_df.tree_id=tree_ids_service_requests

In [ ]:
# Save file with tree ids for damages
service_request_df.to_csv('Data/Intermediate/tree_service_requests_with_id.csv', index=False)

In [37]:
# export id columns too, just in case
tid_service_request_df = pd.DataFrame(service_request_df, columns=["tree_id"])
tid_service_request_df.to_csv('Data/Intermediate/service_request_tree_id.csv', index=False)

### Now merge the street tree census data to branch fall data

In [39]:
# Join the tree fall df to the street tree census df

# First, drop rows without a tree_id from the branch fall df
fall_df = fall_df[fall_df.tree_id.isna()==False]

# now merge the branch fall df to the street tree census df
census_with_falls_df = tree_census_df.merge(fall_df,how="left",left_on="tree_id",right_on="tree_id")

In [47]:
# Finally, save this as a csv so we don't need to do it again
census_with_falls_df.to_csv('Data/Intermediate/tree_data_with_falls.csv', index=False)

### Merge the service request data with the street tree census data
Note the service requests have many potential descriptors (e.g. 'Branch Cracked and Will Fall', 'Tree Leaning/Uprooted') and these will be one hot encoded in the data.  Note that there may be multiple service requests for a tree, but this will only count the presence or absence of the service request type, not the total number.  Also note that only service requests which appeared to influence tree branch falls in EDA are kept in an effort to reduce the number of features in the model.

In [ ]:
# Delete un-needed clomuns
census_with_falls_df.drop(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Complaint Type','Location Type', 'Incident Address',
       'Latitude', 'Longitude','block_id', 'created_at','latitude', 'longitude', 'problems','status'],axis=1,inplace=True )
service_request_df.drop(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Complaint Type','Location Type', 'Incident Address',
       'Latitude', 'Longitude'],axis=1,inplace=True )

In [ ]:
# One hot encode the desired service requests

# First pick the desired service requests
service_request_list = ['Branch Cracked and Will Fall', 'Tree Leaning/Uprooted','Hitting Building','Hitting Power/Phone Lines','Dead Branches in Tree','Blocking Street','Tree Alive - in Poor Condition']

# Loop through the service request list and one hot encode them by looping through the tree_ids in the service request dataframe
# Make a new dataframe, df_temp, from the tree_ids and one hot encoded service requests
service_request_tid_list = service_request_df.tree_id.unique()
df_temp = pd.DataFrame()
df_temp['tree_id']=service_request_tid_list
df_temp.dropna(inplace=True)
df_temp.set_index('tree_id',inplace=True)
for service_request in service_request_list:
    df_temp[service_request]=0
    for tid in service_request_tid_list:
        df_temp.loc[tid,service_request] = service_request_df[service_request_df.tree_id==tid].Descriptor.isin([service_request]).any()

In [ ]:
# Save the one hot encoded service requests temporary dataframe so we don't have to run that loop again
df_temp.to_csv('Data/Intermediate/tree_id_service_requests.csv', index=True)

In [ ]:
# Merge the one hot encoded service requests temporary dataframe with the tree census dataframe
census_with_falls_service_requests_df = census_with_falls_df.merge(df_temp,how="left",left_on="tree_id",right_on="tree_id")

### Now, import and organize the spatial datasets (impervious percentage and building height)
Import the the % impervious cover near the tree (10 m squares) and the building height nearest to the tree, this data was generated in QGIS.

In [ ]:
# Import the impervious % near the tree and closest building height to the tree
census_with_imp_bldghght_df = pd.read_csv('Data/spatialdatasets/data_with_imp_bldghght.csv',index_col='tree_id')

# Drop un-needed columns from the impervious/building height data
census_with_imp_bldghght_df.drop(['address', 'block_id', 'created_at', 'curb_loc', 'guards', 'health',
       'latitude', 'longitude', 'problems', 'root_grate', 'root_other',
       'root_stone', 'sidewalk', 'spc_common', 'status', 'steward', 'tree_dbh',
       'trnk_light', 'trnk_other', 'trnk_wire', 'zipcode'],axis=1,inplace=True)

# Some addresses have multiple building heights, so remove duplicate building heights (note that it isn't too important which one is removed since they are typically very similar)
census_with_imp_bldghght_df = census_with_imp_bldghght_df(subset='tree_id',inplace=True)

### Finally, merge the spatial datasets to the tree census data to create the model input dataframe
Merge the tree census with branch falls and service request dataframe with the impervious percentage and building height dataframe to a model data dataframe containing all data for modeling

In [ ]:
# merge tree, fall, and service request data with building height and impervious percentage
model_data_df = census_with_falls_service_requests_df.merge(census_with_imp_bldghght_df,how='left',left_index=True,right_on='tree_id')

In [ ]:
# Rename the confusing columns of this model dataframe and drop un-needed columns
model_data_df.rename(columns={'rvalue_1':'impervious_pct'},inplace=True)
model_data_df.rename(columns={'Descriptor':'Fall'},inplace=True)
model_data_df.drop('Incident Zip',axis=1,inplace=True)

In [ ]:
# Save the model dataframe
model_data_df.to_csv('Data/Intermediate/model_data.csv', index=False)